Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [86]:
NAME = "Olga Kireeva"
COLLABORATORS = "BAM182"

---

In [87]:
import numpy as np

from numpy.testing import assert_allclose


### Часть I. Постройте отражение Хаусхолдера для вектора.

Дан вектор $\mathbf{x}$ и плоскость, заданная вектором нормали $\mathbf{u}$. Преобразование Хаусхолдера отражает $\mathbf{x}$ относительно плоскости.

Матрица преобразований Хаусхолдера:
$$ \mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T $$

Если даны два вектора $\mathbf{x}$ и $\mathbf{y}$ одинаковой длины, поворот, преобразующий $\mathbf{x}$ в $\mathbf{y}$ называется преобразованием Хаусхолдера с
$$ \mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|} $$

Напишите функцию, преобразующую заданный вектор $\mathbf{x} = (x_1, \dots, x_n)$ в $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$, используя преобразование Хаусхолдера.


In [88]:
def householder(vec):
    """Создает отражение Хаусхолдера, преобразующее 2-ю, 3-ю и т.д. компоненты вектора vec в нули.
    
    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Введенный вектор
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Преобразованный вектор, причем ``outvec[1:]==0`` и ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Ортогональная матрица отражений Хаусхолдера
    """



    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)

    if np.size(vec)==1 and np.linalg.norm(vec) < 1e-8:
      return([0],[[1]])

    x = vec
    y = np.zeros(np.size(vec))
    y[0] = np.linalg.norm(vec)
    u = (x - y) / np.linalg.norm(x - y)
    H = np.eye(np.size(vec)) - 2 * (u[np.newaxis, :].T @ u[np.newaxis, :] )
    return(y, H)

In [89]:
    vec = np.array([1, 2, 3])
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)

    x = vec
    y = np.zeros(np.size(vec))
    y[0] = np.linalg.norm(vec)
    #print(x - y)
    #print(np.linalg.norm(x - y))
    u = (x - y) / np.linalg.norm(x - y)
    #print(u)
    #print(u[np.newaxis, :].T)
    #print(u[np.newaxis, :].T @ u[np.newaxis, :])
    #print(2 * (u[np.newaxis, :].T @ u[np.newaxis, :] ))
    #print(1 - 2 * (u[np.newaxis, :].T @ u[np.newaxis, :]))
    H = np.eye(3) - 2 * (u[np.newaxis, :].T @ u[np.newaxis, :])
    #print(H)
    #print(H @ x)

Протестируйте свою функцию на следующих примерах:

In [90]:
v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(h @ v1, v, atol=1e-14)
assert_allclose(h @ v, v1, atol=1e-14)

assert_allclose(v1[1:], 0, atol=1e-14)

assert_allclose(h @ h.T, np.eye(3), atol=1e-14)


### Part II. Вычисление $\mathrm{QR}$ - разложения матрицы.

Дана прямоугольная $m\times n$ матрица $\mathbf{A}$. Выполните отражение Хаусхолдера матрицы $\mathbf{H}_1$, преобразующее первый столбец матрицы $\mathbf{A}$ (назовем результат $\mathbf{A}^{(1)}$)

$$ 
\mathbf{H}_1 \mathbf{A} %\begin{pmatrix} \times &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ &amp;&amp; \dots&amp;&amp; \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ \end{pmatrix}
%
\equiv \mathbf{A}^{(1)}\;. 
$$

Теперь рассмотрим нижнюю правую часть матрицы $\mathbf{A}^{(1)}$ и выполним преобразование Хаусхолдера, действующее на 2 столбец $\mathbf{A}$:
$$
\mathbf{H}_2 \mathbf{A}^{(1)} % \begin{pmatrix} \times &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; 0 &amp; \times &amp; \dots &amp; \times \\ &amp;&amp; \dots&amp;&amp; \\ 0 &amp; 0 &amp; \times &amp; \dots &amp; \times \\ \end{pmatrix}
%
\equiv \mathbf{A}^{(2)} \;. $$

Повторяя процесс $n$ раз, получим
$$
\mathbf{H}_{n} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;, 
$$

где $\mathbf{R}$ верхняя треугольная матрица. Так как каждая из матриц $\mathbf{H}_k$ ортогональна, таковым будет и их произведение. Обратная от ортогональной также есть матрица ортогональная. Таким образом, алгоритм создает $\mathrm{QR}$ - разложение матрицы $\mathbf{A}$.

Напишите функцию, получающую прямоугольную матрицу $A$ и возвращающую матрицы $Q$ и $R$ --- компоненты $QR$-разложения $A$.


In [91]:
def qr_decomp(a):
    """Вычисляет QR - разложение матрицы.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        Введенная матрица
    
    Returns
    -------
    q : ndarray, shape(m, m)
        Ортогональная матрица
    r : ndarray, shape(m, n)
        Верхняя треугольная матрица
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.testing.assert_allclose(q @ r, a, atol=1e-14)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    count = min(m,n)
    QT = np.eye(m)
    Q = np.eye(m)
    for i in range(0, count):
      v, q = householder(a1[:,0])
      a1=q @ a1
      qa = np.eye(m)
      qa[i:,i:] = q
      QT = qa @ QT
      Q = Q @ (np.transpose(qa))
      a1 = a1[1:,1:]

    R = QT @ a
    return Q, R

In [92]:
a = np.array([[1,2,3],[4,5,6],[7,8,9],[11,12,13]])
q, r = qr_decomp(a)
#print(q @ r)

In [93]:
    a = np.array([[1,2,3],[4,5,6],[7,8,9],[11,12,13]])
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    count = min(m,n)
    QT = np.eye(m)
    Q = np.eye(m)
    for i in range(0,count):
      v1, q = householder(a1[:,0])
      a1 = q @ a1
      qa = np.eye(m)
      qa[i:,i:] = q
      QT = qa @ QT
      Q = Q @ (np.transpose(qa))
      a1 = a1[1:,1:]
    R = QT @ a
   # print(Q @ R)

In [94]:
a1 = np.array([[1,2,3],[4,5,6],[7,8,9]])
Q = np.eye(3)
Qad=np.eye(3)
v1, q = householder(a1[:,0])
Q = q @ Q
Qad = Qad @ (np.transpose(q))
A1 = q @ a1
A1 = A1[1:,1:]
v1, q = householder(A1[:,0])
A2 = q @ A1
qa=np.eye(3)
qa[1:,1:]=q
Q = qa @ Q
Qad=Qad @ (np.transpose(qa))
A2 = A2[1:,1:]
v1, q = householder(A2[:,0])
A3 = q @ A2
qa=np.eye(3)
qa[2:,2:]=q
Q = qa @ Q
Qad = Qad @ (np.transpose(qa))
R = Q @ a1

In [95]:
# можете запустить данную операцию для бооее сжатого вывода: нули вместо `1e-16` и т.д.
np.set_printoptions(suppress=True)

In [96]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
aa = a.copy()

q, r = qr_decomp(a)

# check that `qr_decomp` leaves `a` intact
assert_allclose(a, aa, atol=1e-16)

# тестируем, что Q ортогональна
assert_allclose(q @ q.T, np.eye(5), atol=1e-14)

# проверяем разложение
assert_allclose(q @ r, a, atol=1e-14)

Теперь сравните ваше разложение с разложением, полученным библиотечной функцией (содержащей соответствующие функции библиотеки LAPACK)

In [97]:
from scipy.linalg import qr
qq, rr = qr(a)

print(q,"\n",r)
print(qq,"\n",rr)

assert_allclose(np.dot(qq, rr), a,
                atol=1e-14)

[[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]] 
 [[1.40152769 1.2514379  0.89523615]
 [0.         0.84158252 0.68447942]
 [0.         0.         0.5496046 ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]
[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 
 [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          

Проверьте, согласуются ли ваши матрицы `q` и `r` с `qq` и `rr`. Объясните результат.
Оставьте пояснения в этой ячейке.


### Часть III. Безматричная реализация.

Отметим необычную структуру матрицы Хаусхолдера: матрица поворота $\mathbf{H}$ полностью характеризуется вектором отражения $\mathbf{u}$. Заметим, также, что вычислительная сложность операции отражения матрицы сильно зависит от порядка операций:
$$ \left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A} \qquad \textrm{is } O(m^2 n)\;, $$

тогда как $$ \mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn) $$

Таким образом, следует избегать формирований матриц $\mathbf{H}$. Вместо этого можно сохранять сами векторы отражения $\mathbf{u}$ и производить умножение произвольной матрицы на $\mathbf{Q}^T$, например, как отдельную функцию (класс).

Напишите функцию, выполняющую QR - разложение матрицы без формирования матриц $\mathbf{H}$ и возвращающую матрицу $\mathbf{R}$, а также вектора отражений Хаусхолдера.


Напишите вторую функцию, которая использует вектора отражений, полученных предыдущей функцией, для вычисления произведения $Q^T B$ для заданной матрицы B. Убедитесь, что вы добавили достаточно комментариев, следующих вашим выкладкам. 



In [98]:
def qr_nomatrix(a):
    """Form QR decomposition of `a` via successive Householder reflections.
    
    Parameters
    ----------
    a : ndarray
        Input matrix
    
    Returns
    -------
    R : ndarray
        Upper triangular matrix of the QR decomposition
    
    U : ndarray
        Columns store successive Householder reflectors: `U[j:, j]` stores
        the Householder reflector for reducing the `j`-th column.
        
    See Also
    --------
    householder_apply : apply Householder reflectors stored in `U` to `a`.
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    count = min(m,n)

    QT = np.eye(m)
    Q = np.eye(m)
    U = np.eye(m)

    for i in range(0, count):
      vec = np.asarray(a1[:,0], dtype=float)
      if vec.ndim != 1:
          raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
      if np.size(vec)==1 and np.linalg.norm(vec)<1e-8:
        q = [[1]]
        u = np.asarray([0])
      else:
        x = vec
        y = np.zeros(np.size(vec))
        y[0] = np.linalg.norm(vec)
        u = (x - y) / np.linalg.norm(x - y)
      U[i:, i] = u
      q = np.eye(np.size(vec))-2*(u[np.newaxis, :].T @ u[np.newaxis, :] )
      a1 = q @ a1
      qa = np.eye(m)
      qa[i:,i:] = q
      QT = qa @ QT
      Q = Q @ (np.transpose(qa))
      a1 = a1[1:,1:]
    R = QT @ a
    return R, U
    
def householder_apply(b, uu):
    """Apply the Householder reflectors stored in `uu` to `b`.
    
    The result is equivalent to
    >>> r, q = qr_decomp(a)
    >>> q.T @ b
    
    Parameters
    ----------
    b : ndarray
        Input matrix
    uu : ndarray
        Householder reflectors: `U[j:, j]` is the reflection vector
        for transforming the `j`-th column of `a`.
        
    Returns
    -------
    r : ndarray
        The result of applying the reflectors to `b`. Equivalent to
        computing `q.T @ b`.

    See Also
    --------
    qr_decomp
    
    """
    a1 = np.array(b, copy=True, dtype=float)
    m, n = a1.shape
    count = min(m,n)
    QT = np.eye(m)
    Q = np.eye(m)
    for i in range(0,count):
      u = U[i:, i]
      vec = np.asarray(a1[:,0], dtype=float)
      q = np.eye(np.size(vec))-2*(u[np.newaxis, :].T @ u[np.newaxis, :] )
      a1 = q @ a1
      qa = np.eye(m)
      qa[i:,i:] = q
      QT = qa @ QT
      Q = Q @ (np.transpose(qa))
      a1 = a1[1:,1:]
    R=QT@b
    return R

In [99]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
R1, U = qr_nomatrix(a)
R2 = householder_apply(a, U)

In [100]:
rndm = np.random.RandomState(1223)

a = rndm.uniform(size=(5, 5))
R1, U = qr_nomatrix(a)
R2 = householder_apply(a, U)
R_lib = qr(a)[1]
assert_allclose(R1, R2, atol=1e-14)


In [101]:
# Check consistency with the scipy library decomposition. Allow for sign differences

conds = [np.allclose(R2[i, :], R_lib[i, :], atol=1e-14) or
         np.allclose(R2[i, :], -R_lib[i, :], atol=1e-14) for i in range(5)]
assert False not in conds


In [102]:
# More testing here, keep this cell intact
